## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre:Santiago
- Apellido:Vicentela
- Legajo:1729093

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
#Importo el CSV y reviso que se haya importado bien

ruta_arhivo=r"C:\Users\SANTIAGO\Downloads\TP2_airbnb_london.csv"
london= pd.read_csv(ruta_arhivo)
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london"

In [4]:

# Imprimo las dimensiones del dataset
print(f"Dimensiones del dataset antes de eliminar la primera columna: {london.shape}")


# Elimino la primera columna (independientemente de su nombre)
london = london.iloc[:, 1:]

# Imprimo las nuevas dimensiones del dataset
print(f"Dimensiones del dataset después de eliminar la primera columna: {london.shape}")

# Guardo el dataset modificado bajo el nombre "london"
london.to_csv('london', index=False)

print("El dataset ha sido guardado bajo el nombre 'london' sin la primera columna.")

london.head()

Dimensiones del dataset antes de eliminar la primera columna: (51827, 9)
Dimensiones del dataset después de eliminar la primera columna: (51827, 8)
El dataset ha sido guardado bajo el nombre 'london' sin la primera columna.


,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Haringey,Private room,35,1,131,1.71,4,262
1,Ealing,Private room,70,2,2,0.07,6,62
2,Islington,Private room,45,1,14,0.14,2,364
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [5]:

# Verifico si hay valores nulos
nulos_totales = london.isnull().sum().sum()



if nulos_totales > 0:
    print(f"El dataset contiene {nulos_totales} valores nulos.")
    
   
    london.fillna(0, inplace=True)
    print("Todos los valores nulos han sido llenados con 0.")
else:
    print("El dataset no contiene valores nulos.")

    
    

london.to_csv('london', index=False)

print("El dataset ha sido guardado bajo el nombre 'london' sin valores nulos.")

El dataset no contiene valores nulos.
El dataset ha sido guardado bajo el nombre 'london' sin valores nulos.


##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [6]:
# Mostrar los tipos de datos de las variables "neighbourhood" y "room_type"
print("Tipo de dato de la columna 'neighbourhood':", london['neighbourhood'].dtype)
print("Tipo de dato de la columna 'room_type':", london['room_type'].dtype)


Tipo de dato de la columna 'neighbourhood': object
Tipo de dato de la columna 'room_type': object


##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [7]:
# Genero las variables dummies solo para 'neighbourhood'
neigh_dummies = pd.get_dummies(london['neighbourhood'], drop_first=True)



Imprimir las dimensiones y primeras lineas de estas dummies

In [8]:
# Imprimo las dimensiones de las dummies
print(f"Dimensiones de las variables dummies: {neigh_dummies.shape}")

Dimensiones de las variables dummies: (51827, 32)


In [9]:
# Imprimo las primeras filas de las dummies generadas
print("\nPrimeras filas de las variables dummies:")
neigh_dummies.head()


Primeras filas de las variables dummies:


,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,Greenwich,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Genero las variables dummies solo para 'room_type'
room_dummies = pd.get_dummies(london['room_type'], drop_first=True)




Imprimir las dimensiones y primeras lineas de estas dummies

In [11]:
# Imprimo las dimensiones de las dummies
print(f"Dimensiones de las dummies de 'room_type': {room_dummies.shape}")

Dimensiones de las dummies de 'room_type': (51827, 2)


In [12]:
# Imprimo las primeras líneas de las dummies
print("\nPrimeras líneas de las dummies de 'room_type':")
room_dummies.head()


Primeras líneas de las dummies de 'room_type':


,Private room,Shared room
0,1,0
1,1,0
2,1,0
3,0,0
4,1,0


Joinear las variables dummies al dataset



In [13]:
# Concatenar las dummies con el dataset 'london'
london = pd.concat([london, room_dummies, neigh_dummies], axis=1)

# Imprimo las dimensiones del dataset actualizado
print(f"Dimensiones del dataset 'london' después de añadir las dummies: {london.shape}")


print("\nPrimeras líneas del dataset 'london' después de añadir las dummies:")


london.head()

Dimensiones del dataset 'london' después de añadir las dummies: (51827, 42)

Primeras líneas del dataset 'london' después de añadir las dummies:


,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Private room,Shared room,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,Haringey,Private room,35,1,131,1.71,4,262,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Ealing,Private room,70,2,2,0.07,6,62,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Islington,Private room,45,1,14,0.14,2,364,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Barnet,Private room,29,5,114,1.13,3,285,1,0,...,0,0,0,0,0,0,0,0,0,0


##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [14]:

y = np.array(london[["price"]])
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)


# Imprimo las dimensiones de 'x' y 'y' para verificar
print(f"Dimensiones de 'x' (variables independientes): {x.shape}")
print(f"Dimensiones de 'y' (variable dependiente): {y.shape}")

Dimensiones de 'x' (variables independientes): (51827, 39)
Dimensiones de 'y' (variable dependiente): (51827, 1)


##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [15]:
from sklearn.model_selection import train_test_split

# Divido los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)



# Imprimo las dimensiones de los conjuntos resultantes
print(f"Dimensiones de 'x_train': {x_train.shape}")
print(f"Dimensiones de 'x_test': {x_test.shape}")
print(f"Dimensiones de 'y_train': {y_train.shape}")
print(f"Dimensiones de 'y_test': {y_test.shape}")


Dimensiones de 'x_train': (36278, 39)
Dimensiones de 'x_test': (15549, 39)
Dimensiones de 'y_train': (36278, 1)
Dimensiones de 'y_test': (15549, 1)


##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [16]:
scaler = preprocessing.StandardScaler().fit(x_train)

In [17]:
xtrain_scal = scaler.transform(x_train)
print(f"Dimensiones de 'xtrain_scal': {xtrain_scal.shape}")

Dimensiones de 'xtrain_scal': (36278, 39)


In [18]:
xtest_scal = scaler.transform(x_test)
print(f"Dimensiones de 'xtest_scal': {xtest_scal.shape}")


Dimensiones de 'xtest_scal': (15549, 39)


##### **Objetivo 9**
Definir un modelo de regresion lineal standard y entrenarlo para nuestro dataset (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

Obtener las predicciones y el "mean_squared_error" para el test set.

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error



# Defino el modelo de regresión lineal
model = LinearRegression()



# Entreno el modelo con los datos de entrenamiento escalados
model.fit(xtrain_scal, y_train)




# Realizo predicciones en el conjunto de prueba escalado
y_pred = model.predict(xtest_scal)




# Calculo el error cuadrático medio (Mean Squared Error) para el conjunto de prueba
mse = mean_squared_error(y_test, y_pred)


# Mostrar las predicciones y el MSE
print(f"Predicciones del modelo:\n{y_pred}\n")
print(f"Error cuadrático medio (MSE) en el test set: {mse:.4f}")


Predicciones del modelo:
[[128.44001398]
 [116.59497247]
 [ 97.50645509]
 ...
 [ 35.53742982]
 [ 38.82674032]
 [ 53.11505868]]

Error cuadrático medio (MSE) en el test set: 1520.0564


##### **Objetivo 10**
Definir un modelo de regresion SVM Regressor (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) e implementar GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [72]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error



# Defino el modelo SVM Regressor
svr = SVR()


# Defino los parámetros para la búsqueda en cuadrícula
param_grid = {
    'kernel': ['linear', 'rbf'],  # Dos posibles valores para el parámetro "kernel"
    'C': [0.1, 1, 10],            # Tres posibles valores para "C"
    'gamma': [0.01, 0.1, 1]       # Tres posibles valores para "gamma"
}



# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, refit=True)


# Ajusto el modelo
grid_search.fit(xtrain_scal, y_train.ravel())


# Obtengo los mejores parámetros
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_


# Realizo predicciones
predictions = best_model.predict(xtest_scal)


# Calculo el Error Cuadrático Medio
mse = mean_squared_error(y_test, predictions)


# Imprimir resultados
print(f"Mejores parámetros: {best_params}")
print(f"Error cuadrático medio (MSE) en el test set: {mse:.4f}")


Mejores parámetros: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Error cuadrático medio (MSE) en el test set: 1557.8427


##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set del punto anterior

In [73]:
print(f"Mejores hiperparámetros: {best_params}")


Mejores hiperparámetros: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}


In [74]:
print(f"Error cuadrático medio (MSE) en el test set: {mse:.4f}")

Error cuadrático medio (MSE) en el test set: 1557.8427


##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error



# Defino el modelo SVM Regressor
svr = SVR()



# Defino nuevos valores para los parámetros de búsqueda
param_grid = {
    'kernel': ['rbf', 'poly'],  
    'C': [5, 50, 100],                  
    'gamma': [0.001,0.005,5]       
}


# Configuro GridSearchCV
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, refit=True)



# Ajusto el modelo
grid_search.fit(xtrain_scal, y_train.ravel())


# Obtengo los mejores parámetros
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_


# Realizo las predicciones
predictions = best_model.predict(xtest_scal)

# Calculo el Error Cuadrático Medio
mse = mean_squared_error(y_test, predictions)


# Imprimir resultados
print(f"Mejores parámetros: {best_params}")
print(f"Error cuadrático medio (MSE) en el test set: {mse:.4f}")



Conclusiones:


Al realizar la búsqueda de hiperparámetros mediante GridSearchCV, obtuvimos los siguientes mejores parámetros:

C: 10
gamma: 0.01
kernel: 'rbf'

Con estos hiperparámetros, logramos un error cuadrático medio (MSE) de 1557.8427 en el test set.


Esto significa que, al optimizar el modelo SVM usando un kernel radial base (rbf), con un valor de penalización C más alto (10), y un gamma pequeño (0.01), el modelo fue capaz de mejorar su capacidad de ajuste. 
